In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression

In [2]:
def count_metric(y_actual, y_predicted):
    y_actual_log = np.log(y_actual + 1)
    y_predicted_log = np.log(y_predicted + 1)
    rms = sqrt(mean_squared_error(y_actual_log, y_predicted_log))
    return rms

def rmse(y_act,y_pred):
    rms=sqrt(mean_squared_error(y_act,y_pred))
    return rms
def loss(x):
    return np.sum(np.square((np.dot(x, m) - y)))

## Actual output

In [3]:
Xtrain = pd.read_csv('Data/train_data_hhp.csv')
Xtest = pd.read_csv('Data/test_data_hhp.csv')

Ytrain=Xtrain['DaysInHospital']
Ytest=Xtest['DaysInHospital']

## Input partial output from XGB , RandomForest (classifier and regressor)

In [4]:
#Regressor XGB _scaled
Y2_pred_XGBregr_Scaled = pd.read_csv('Data/Y2_pred_XGBregr_scaled.csv')
Y3_pred_XGBregr_Scaled = pd.read_csv('Data/Y3_pred_XGBregr_scaled.csv')
#Regressor XGB _ notscaled
Y2_pred_XGBregr_notScaled = pd.read_csv('Data/Y2_pred_XGBregr_notScaled.csv')
Y3_pred_XGBregr_notScaled = pd.read_csv('Data/Y3_pred_XGBregr_notScaled.csv')

Y2_pred_XGBregr_Scaled.drop(['Unnamed: 0'],axis=1, inplace=True)
Y3_pred_XGBregr_Scaled.drop(['Unnamed: 0'],axis=1, inplace=True)
Y2_pred_XGBregr_notScaled.drop(['Unnamed: 0'],axis=1, inplace=True)
Y3_pred_XGBregr_notScaled.drop(['Unnamed: 0'],axis=1, inplace=True)

#Regressor RF _scaled
Y2_pred_RFregr_Scaled = pd.read_csv('Data/Y2_pred_RFregr_scaled.csv')
Y3_pred_RFregr_Scaled = pd.read_csv('Data/Y3_pred_RFregr_scaled.csv')
#Regressor RF _ notscaled
Y2_pred_RFregr_notScaled = pd.read_csv('Data/Y2_pred_RFregr_notScaled.csv')
Y3_pred_RFregr_notScaled = pd.read_csv('Data/Y3_pred_RFregr_notScaled.csv')


Y3_pred_RFregr_Scaled.drop(['Unnamed: 0'],axis=1, inplace=True)
Y2_pred_RFregr_Scaled.drop(['Unnamed: 0'],axis=1, inplace=True)
Y3_pred_RFregr_notScaled.drop(['Unnamed: 0'],axis=1, inplace=True)
Y2_pred_RFregr_notScaled.drop(['Unnamed: 0'],axis=1, inplace=True)

In [5]:
#Regressor LGBM _scaled1
Y2_pred_LGBMregr_Scaled1 = pd.read_csv('Data/Y2_pred_LGBM_scaled1.csv')
Y3_pred_LGBMregr_Scaled1 = pd.read_csv('Data/Y3_pred_LGBM_scaled1.csv')
Y2_pred_LGBMregr_Scaled1.drop(['Unnamed: 0'],axis=1, inplace=True)
Y3_pred_LGBMregr_Scaled1.drop(['Unnamed: 0'],axis=1, inplace=True)

In [6]:
logYtrain=np.log(Ytrain+1)
logYtest=np.log(Ytest+1)

# Ensemble

## Combine the candidates -  Random Forest (regressor and classifier), XGB (regressor and classifier)

In [7]:
train= pd.concat([Y2_pred_LGBMregr_Scaled1, 
                 Y2_pred_RFregr_Scaled, 
                 Y2_pred_RFregr_notScaled,
                 Y2_pred_XGBregr_Scaled,
                Y2_pred_XGBregr_notScaled
                ], axis=1)

test= pd.concat([Y3_pred_LGBMregr_Scaled1, 
                 Y3_pred_RFregr_Scaled, 
                Y3_pred_RFregr_notScaled,
                 Y3_pred_XGBregr_Scaled, 
                Y3_pred_XGBregr_notScaled
                ], axis=1)
train=np.log(train+1)
test=np.log(test+1)

### XGBregressor

In [8]:
XGregr = XGBRegressor()

#### Ensemble on log scale

In [9]:
XGregr.fit(train, logYtrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [10]:
logY_pred_test = XGregr .predict(test)
logY_pred_train = XGregr .predict(train)

print('RSME on traing set',rmse(logYtrain, logY_pred_train),
        'RSME on testing set',rmse(logYtest, logY_pred_test),
     )

RSME on traing set 0.2715083233007804 RSME on testing set 0.4746792579453372


### Linear regresion

#### Ensemble on log scale

In [11]:
reg = LinearRegression()
reg.fit(train, logYtrain)
logY_pred_test = reg.predict(test)
logY_pred_train = reg .predict(train)

print('RSME on traing set',rmse(logYtrain, logY_pred_train),
        'RSME on testing set',rmse(logYtest, logY_pred_test)
     )

RSME on traing set 0.28042987581259277 RSME on testing set 0.4779778154351217


### Linear combination

#### Linear regression (on log scale)

In [14]:
reg = LinearRegression()
reg.fit(train, logYtrain)
logY_pred_test = reg.predict(test)
logY_pred_train = reg .predict(train)

print('RSME on traing set',rmse(logYtrain, logY_pred_train),
        'RSME on testing set',rmse(logYtest, logY_pred_test)
     )

RSME on traing set 0.28042987581259277 RSME on testing set 0.4779778154351217


### Stack

In [ ]:
def Stacking(model,train,y,test,n_fold):
   folds=StratifiedKFold(n_splits=n_fold,random_state=1)
   test_pred=np.empty((test.shape[0],1),float)
   train_pred=np.empty((0,1),float)
   for train_indices,val_indices in folds.split(train,y.values):
      x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
      y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

      model.fit(X=x_train,y=y_train)
      train_pred=np.append(train_pred,model.predict(x_val))
      test_pred=np.append(test_pred,model.predict(test))
    return test_pred.reshape(-1,1),train_pred

## Combinatoric candidates

In [15]:
x_train=(train.values).T
x_test=(test.values).T
y_train=(logYtrain.values).T
y_test=(logYtest.values).T

m =x_train
y = y_train

from scipy.optimize import minimize, rosen, rosen_der

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x) - 1.0})

x0 = np.zeros(m.shape[0])
res = minimize(loss, x0, method='SLSQP', constraints=cons,
               bounds=[(0, np.inf) for i in range(m.shape[0])], options={'disp': True})

print('weight',res.x)
Y_pred_test=(res.x*x_test.T).sum(axis=1)
Y_pred_train=(res.x*x_train.T) .sum(axis=1)
print('RSMLE on traing set',count_metric(y_train.T, Y_pred_train.T),
        'RSMLE on testing set',count_metric(y_test.T, Y_pred_test.T),
     )

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 6940.832886089734
            Iterations: 10
            Function evaluations: 52
            Gradient evaluations: 6
weight [0.00000000e+00 8.12696322e-01 1.87313257e-01 5.53624935e-07
 6.17110627e-07]
RSMLE on traing set 0.18303287657454095 RSMLE on testing set 0.29120926614744774


In [ ]:
Y_pred_test